In [4]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import evaluate
import torch
import numpy as np

In [6]:
dataset = load_dataset("imdb")

small_train = dataset["train"].shuffle(seed=42).select(range(1000))  # 1k samples
small_test = dataset["test"].shuffle(seed=42).select(range(250))     # 250 samples

In [7]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize(batch):
    return tokenizer(
        batch["text"],
        padding="max_length",  # Pad shorter sequences
        truncation=True,       # Cut sequences >512 tokens
        max_length=512         # BERT's max input length
    )

tokenized_dataset_train = small_train.map(tokenize, batched=True)
tokenized_dataset_test = small_test.map(tokenize, batched=True)

Map: 100%|██████████| 250/250 [00:00<00:00, 442.17 examples/s]


In [8]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

e:\IITB\Learner Space 2025\Intro-to-ML-and-NLP\hf_env\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ANITA\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at be

In [10]:
metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        **metric.compute(predictions=predictions, references=labels),
        **f1_metric.compute(predictions=predictions, references=labels)
    }

In [15]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=2,
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_test,
    compute_metrics=compute_metrics,
)

In [16]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.331382,0.872000,0.855856
2,No log,0.312415,0.872000,0.869919


e:\IITB\Learner Space 2025\Intro-to-ML-and-NLP\hf_env\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=126, training_loss=0.41880531916542657, metrics={'train_runtime': 4047.0763, 'train_samples_per_second': 0.494, 'train_steps_per_second': 0.031, 'total_flos': 526222110720000.0, 'train_loss': 0.41880531916542657, 'epoch': 2.0})

In [17]:
model.save_pretrained("./fine_tuned_bert")
tokenizer.save_pretrained("./fine_tuned_bert")

('./fine_tuned_bert\\tokenizer_config.json',
 './fine_tuned_bert\\special_tokens_map.json',
 './fine_tuned_bert\\vocab.txt',
 './fine_tuned_bert\\added_tokens.json')

In [22]:
from transformers import pipeline

sample_text = "This movie was not fantastic!"
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer)
print("Sample prediction:", pipe(sample_text)) # Label_1: Positive, Label_0: Negative

Device set to use cpu


Sample prediction: [{'label': 'LABEL_0', 'score': 0.9082422256469727}]
